In [ ]:
import os
import sys
import torch
import glob
import csv
import pandas as pd
import numpy as np
import nilearn.datasets
import nilearn.connectome
import pathlib
from nilearn.input_data import NiftiMasker
from nilearn import datasets, plotting, image
from csv import writer

sys.path.append(os.path.join(".."))
sys.path.append('../src')
import gcn_windows_dataset
import graph_construction
import gcn_model
import visualization

## Fetching Haxby dataset

In this notebook, all the models are trained on the *Haxby* dataset.
We could easily removed resting state tasks at the time of selecting data. That would give us higher accuracy results.

__*X = masker.fit_transform(func_file)[nonrest_task_mask]* \
*y = labels['labels'][nonrest_task_mask]*__

In [ ]:
# We are fetching the data for subject 4
data_dir = os.path.join('..', 'data')
sub_no = 4
haxby_ds = datasets.fetch_haxby(subjects=[sub_no], fetch_stimuli=True, data_dir=data_dir)

func_file = haxby_ds.func[0]

# Standardizing
mask_vt_file = haxby_ds.mask_vt[0]
masker = NiftiMasker(mask_img=mask_vt_file, standardize=True)

labels = pd.read_csv(haxby_ds.session_target[0], sep=" ")

# Selecting data
X = masker.fit_transform(func_file)
y = labels['labels']

categories = y.unique()
print(categories)
print(y.shape)
print(X.shape)

# Graph Convolutional Networks (GCN)

### Model pipeline:
- Takes a short series of fMRI volumes as input.
- Maps the fMRI signals onto a predefined brain graph.
- Propagates brain dynamics information on inter-connected brain regions & networks.
- Generates task-specific representations of recorded brain activities. 
- Predicts the corresponding task states.


<img src="GCN_pipeline.png" width=850 height=420 />

## Data paths

In [ ]:
proc_path = os.path.join(data_dir, 'haxby_proc/')
concat_path = os.path.join(data_dir, 'haxby_concat/')
conn_path = os.path.join(data_dir, 'haxby_connectomes/')
split_path = os.path.join(data_dir, 'haxby_split_win/')
    
# delete the contents of a folder to avoid inconsistency
old_files = glob.glob(concat_path + '/*')
for f in old_files:
    os.remove(f)    
if os.path.exists(split_path):
    files = glob.glob(os.path.join(split_path, "*"))
    for f in files:
        os.remove(f)

## Data processing

For the GCN model in order to run the model on different sizes of input, we will concatenate bold data of the same stimuli and save it in a single file.

In [ ]:
old_dirContents = os.listdir(concat_path)
print(old_dirContents)

concat_bold_files = []
if (len(old_dirContents) == 0 or len(old_dirContents) == 1):    
    if ((len(X)) == len(y)):
        
        for i in range(0,len(y)):
            label = y[i]
            concat_bold_files = X[i:i+1]
            concat_file_name = concat_path + '{}_concat_fMRI.npy'.format(label)
            file = pathlib.Path(concat_file_name)
            
            if file.exists ():
                concat_file = np.load(concat_file_name, allow_pickle = True)
                concat_file = np.concatenate((concat_file, concat_bold_files), axis = 0)
                np.save(concat_file_name, concat_file)
            else:
                np.save(concat_file_name, concat_bold_files)
            
else:
    print('Folder is Not Empty')

In [ ]:
with open(concat_path + 'phenotypic_data.tsv', 'wt') as out_file:
    
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['label'])
    
    for category in categories: 
        tsv_writer.writerow([category])

## Generating connectomes

In [ ]:
# Estimating connectomes
corr_measure = nilearn.connectome.ConnectivityMeasure(kind="correlation")
conn = corr_measure.fit_transform([X])[0]
np.save(os.path.join(conn_path, 'conn_subj{}.npy'.format(sub_no)), conn)

## Time windows

Different lengths for our input data can be selected. 
In this example we will continue with *window_length = 1*, which means each input file will have a length equal to just one Repetition Time (TR). 

In [ ]:
window_length = 1

# Path for saving the files
pheno_file = os.path.join(concat_path, 'phenotypic_data.tsv')
processed_bold_files = sorted(glob.glob(concat_path + '/*.npy'))
conn_files = sorted(glob.glob(conn_path + '/*.npy'))
out_file = os.path.join(split_path, '{}_{:04d}.npy')
out_csv = os.path.join(split_path, 'labels.csv')

Now we are going to split bold input files to the desired windows lenght, then we will also create a csv file that will contain label for each splited data.

In [ ]:
dic_labels = {'rest':0,'face':1,'chair':2,'scissors':3,'shoe':4,'scrambledpix':5,'house':6,'cat':7,'bottle':8}
label_df = pd.DataFrame(columns=['label', 'filename'])

for proc_bold in processed_bold_files:
    
    ts_data = np.load(proc_bold)
    ts_duration = len(ts_data)

    ts_filename = os.path.basename(proc_bold)
    ts_label = ts_filename.split('_', 1)[0]

    valid_label = dic_labels[ts_label]
    
    # Split the timeseries
    rem = ts_duration % window_length
    n_splits = int(np.floor(ts_duration / window_length))

    ts_data = ts_data[:(ts_duration-rem), :]   
    
    for j, split_ts in enumerate(np.split(ts_data, n_splits)):
        ts_output_file_name = out_file.format(ts_filename, j)

        split_ts = np.swapaxes(split_ts, 0, 1)
        np.save(ts_output_file_name, split_ts)
        curr_label = {'label': valid_label, 'filename': os.path.basename(ts_output_file_name)}
        label_df = label_df.append(curr_label, ignore_index=True)
    
label_df.to_csv(out_csv, index=False)        

## Split dataset

Here we will split the processed data into three diferent sets for train, validation, and test.

In [ ]:
random_seed = 0

train_dataset = gcn_windows_dataset.TimeWindowsDataset(
    data_dir=split_path
    , partition="train"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True,
    shuffle = True)

valid_dataset = gcn_windows_dataset.TimeWindowsDataset(
    data_dir=split_path
    , partition="valid"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True, 
    shuffle = True)

test_dataset = gcn_windows_dataset.TimeWindowsDataset(
    data_dir=split_path
    , partition="test"
    , random_seed=random_seed
    , pin_memory=True
    , normalize=True, 
    shuffle = True)

print("train dataset: {}".format(train_dataset))
print("valid dataset: {}".format(valid_dataset))
print("test dataset: {}".format(test_dataset))

In [ ]:
torch.manual_seed(random_seed)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_generator = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_generator = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
train_features, train_labels = next(iter(train_generator))
print(f"Feature batch shape: {train_features.size()}; mean {torch.mean(train_features)}")
print(f"Labels batch shape: {train_labels.size()}; mean {torch.mean(torch.Tensor.float(train_labels))}")

## Getting connectomes
We will get the average connectome with its k-nearest neighbors, in this model *k = 8* is considered.

In [ ]:
connectomes = []
for conn_file in conn_files:
      connectomes += [np.load(conn_file)]

## Model definition

- __6__ graph convolutional layers with __32 graph filters__  at each layer
- followed by a __global average pooling__ layer & __2 fully connected__ layers 
- k-NN graph was made using __8 nodes__ with the highest connectivity

In [ ]:
graph = graph_construction.make_group_graph(connectomes, self_loops=False, 
                                            k=8, symmetric=True)
# Create model
gcn = gcn_model.GCN(graph.edge_index, graph.edge_attr, 
                           n_timepoints=window_length)
gcn

## Train and evaluating the model

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)    

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * dataloader.batch_size #Loic

        correct = (pred.argmax(1) == y).type(torch.float).sum().item()
        correct /= X.shape[0]
        print(f"#{batch:>5};\ttrain_loss: {loss:>0.3f};\ttrain_accuracy:{(100*correct):>5.1f}%\t\t[{current:>5d}/{size:>5d}]")

def valid_test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model.forward(X)
            loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss /= size
    correct /= size

    return loss, correct

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gcn.parameters(), lr=1e-4, weight_decay=5e-4)

epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}/{epochs}\n-------------------------------")
    train_loop(train_generator, gcn, loss_fn, optimizer)
    loss, correct = valid_test_loop(valid_generator, gcn, loss_fn)
    print(f"Valid metrics:\n\t avg_loss: {loss:>8f};\t avg_accuracy: {(100*correct):>0.1f}%")

## Rasults

In [ ]:
loss, correct = valid_test_loop(test_generator, gcn, loss_fn) 
print(f"Test metrics:\n\t avg_loss: {loss:>f};\t avg_accuracy: {(100*correct):>0.1f}%")

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# multilayer perceptron (MLP)

- with 2 dense layers

In [ ]:
import matplotlib.pyplot as plt
import keras
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from nilearn.plotting import plot_anat, show, plot_stat_map, plot_matrix

In [ ]:
# Encoding the string to numerical values. (ML Algorithm can only work on numbers and not on string)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

# reshapeing y
temp = np.reshape(y, (len(y),1))
y = temp

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
y = pd.DataFrame(enc.fit_transform(y).toarray())
y

## Split dataset
We will shuffle and split the data into training and test sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 0)

#standarize features caling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initializing the model

In [ ]:
# number of unique conditions that we have
mlp_classifier = Sequential()

# Adding the input layer and the first hidden layer
mlp_classifier.add(Dense(338 , input_dim = 675, kernel_initializer="uniform", activation = 'relu'))

# Adding the second hidden layer
mlp_classifier.add(Dense(169, kernel_initializer="uniform", activation = 'relu'))

# Using softmax at the end, lenght of categories shows the number of labels we have
mlp_classifier.add(Dense(len(categories), activation = 'softmax'))

In [ ]:
mlp_classifier.summary()

In [ ]:
# Compiling the model
mlp_classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Fitting the model on the Training set
history = mlp_classifier.fit(X_train, y_train, batch_size = 10,
                             epochs = 10, validation_split = 0.1)

In [ ]:
plot_history = visualization.classifier_history (history, 'MLP ')

In [ ]:
# Making the predictions and evaluating the model
y_pred = mlp_classifier.predict(X_test)
y_pred = (y_pred > 0.5)

print ('mean accuracy score:', np.round(accuracy_score(y_test.values.argmax(axis = 1), 
                                                       y_pred.argmax(axis=1), normalize = True, 
                                                       sample_weight=None),2))

# Confusion matrix
cm_mlp = confusion_matrix(y_test.values.argmax(axis = 1), y_pred.argmax(axis=1))
model_conf_matrix = cm_mlp.astype('float') / cm_mlp.sum(axis = 1)[:, np.newaxis]

visualization.conf_matrix(model_conf_matrix, 
                          categories, 
                          title='MLP decoding results on Haxby')

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Exercises